In [3]:
import torch
from gangen.cdcgan import Discriminator, Generator

In [4]:
discriminator = Discriminator()
generator = Generator()

print(discriminator)

Discriminator(
  (image_features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv2d(32, 32, kernel_size=(4, 4), stride=(4, 4), bias=False)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Flatten(start_dim=1, end_dim=-1)
  )
  (class_emb): Embedding(43, 32)
  (unconditioned): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
import os

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

In [ ]:
data_dir = '/home/takayuki/Desktop/sem6/DL/mini_proj_TRAFFIC/data/'
train_dir = os.path.join(data_dir, 'preprocessed', 'Train')

data_tf = transforms.Compose([
    transforms.RandomAffine(degrees=10, translate=(0.08, 0.08), scale=(0.9, 1.1), shear=5),
    transforms.Resize((32, 32)),
    
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
   
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))], p=0.3),

    transforms.ToTensor(),

    transforms.RandomErasing(p=0.2, scale=(0.01, 0.04), ratio=(0.5, 2.0), value=0), # value=0 for black box

    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


full_dataset = datasets.ImageFolder(root = train_dir, transform=data_tf)